# Toggle_Switch

In [ ]:
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Reaction, Parameter, RateRule, AssignmentRule, FunctionDefinition
from gillespy2 import EventAssignment, EventTrigger, Event
from gillespy2 import SSACSolver
# from gillespy2 import VariableSSACSolver
# from gillespy2 import BasicTauLeapingSolver
# from gillespy2 import BasicTauHybridSolver
# from gillespy2 import BasicODESolver

In [ ]:
class Toggle_Switch(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Toggle_Switch")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="alpha1", expression=10))
        self.add_parameter(Parameter(name="alpha2", expression=10))
        self.add_parameter(Parameter(name="beta", expression=2))
        self.add_parameter(Parameter(name="gamma", expression=2))
        self.add_parameter(Parameter(name="mu", expression=1))

        # Species
        self.add_species(Species(name="A", initial_value=2, mode="discrete"))
        self.add_species(Species(name="B", initial_value=2, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="cu", reactants={}, products={'A': 1}, propensity_function="alpha1/(1+pow(B, beta))"))
        self.add_reaction(Reaction(name="cv", reactants={}, products={'B': 1}, propensity_function="alpha2/(1+pow(A, gamma))"))
        self.add_reaction(Reaction(name="du", reactants={'A': 1}, products={}, rate=self.listOfParameters["mu"]))
        self.add_reaction(Reaction(name="dv", reactants={'B': 1}, products={}, rate=self.listOfParameters["mu"]))

        # Timespan
        self.timespan(np.linspace(0, 250, 251))

In [ ]:
model = Toggle_Switch()

In [ ]:
solver = SSACSolver(model=model)

In [ ]:
def configure_simulation():
# the minimum number of trajectoies needs to be at least 30
    kwargs = {
        "solver":solver,
        "show_labels":False,
        "number_of_trajectories":100,
        "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06}
    }
    return kwargs

In [ ]:
from tsfresh.feature_extraction.settings import MinimalFCParameters
from sciope.utilities.priors import uniform_prior
from sciope.utilities.summarystats import auto_tsfresh
from sciope.utilities.distancefunctions import naive_squared
from sciope.inference.abc_inference import ABC
from sklearn.metrics import mean_absolute_error
from dask.distributed import Client

In [ ]:
kwargs = configure_simulation()

# Define simulator function
def set_model_parameters(params, model):
    """para,s - array, need to have the same order as
    model.listOfParameters """
    for e, (pname, p) in enumerate(model.listOfParameters.items()):
        model.get_parameter(pname).set_expression(params[e])
    return model

# Here we use the GillesPy2 Solver
def simulator(params, model):
    model_update = set_model_parameters(params, model)
    
    res = model_update.run(**kwargs)
    tot_res = np.asarray([x.T for x in res]) # reshape to (N, S, T)
    tot_res = tot_res[:,1:, :] # should not contain timepoints

    return tot_res

# Wrapper, simulator function to abc should should only take one argument (the parameter point)
def simulator2(x):
    return simulator(x, model=model)

In [ ]:
# Set up the prior
default_param = np.array(list(model.listOfParameters.items()))[:,1] # take default from mode 1 as reference

bound = []
for exp in default_param:
    bound.append(float(exp.expression))

# Set the bounds
bound = np.array(bound)
dmin = bound * 0.1
dmax = bound * 2.0

# Here we use uniform prior
uni_prior = uniform_prior.UniformPrior(dmin, dmax)

In [ ]:
# generate some fixed(observed) data based on default parameters of the model
fixed_data = model.run(**kwargs)

In [ ]:
# Reshape the data to (n_points,n_species,n_timepoints) and remove timepoints array
fixed_data = np.asarray([x.T for x in fixed_data])
fixed_data = fixed_data[:,1:, :]

In [ ]:
# Function to generate summary statistics
summ_func = auto_tsfresh.SummariesTSFRESH()

# Distance
ns = naive_squared.NaiveSquaredDistance()

# Start abc instance
abc = ABC(fixed_data, sim=simulator2, prior_function=uni_prior, summaries_function=summ_func.compute, distance_function=ns)

In [ ]:
c = Client()
c

In [ ]:
# First compute the fixed(observed) mean
abc.compute_fixed_mean(chunk_size=2)

In [ ]:
res = abc.infer(num_samples=100, batch_size=10, chunk_size=2)

In [ ]:
mae_inference = mean_absolute_error(bound, abc.results["inferred_parameters"])